Grupo 5A: Nicolás Camousseight / Camila Maire




# Red convolucional para detección temprana de supernovas

In [ ]:
#Librerías

import pickle as pkl
import os
import sys
import time
import numpy as np
import pandas as pd
import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, make_scorer, accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split

import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, TensorDataset
#import tensorflow as tf
#from tensorflow import keras
#from tensorflow.keras import layers, models
#from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from tensorflow.keras.optimizers import Adam

## Red convolucional

### Accedemos a la data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Función para cargar el contenido del archivo .pkl
def load_pickle_data(filename):
    with open(filename, 'rb') as f:
        data = pkl.load(f)
    return data

# Cargar el archivo
filename = '/content/drive/My Drive/stamps.pkl'
data_procesada = load_pickle_data(filename)

# Visualizar el contenido
print('keys de pkl', data_procesada.keys())
print('keys de Test',data_procesada['Validation'].keys())
data_procesada['Validation']['labels']

keys de pkl dict_keys(['Train', 'Validation', 'Test'])
keys de Test dict_keys(['images', 'labels', 'features'])


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3,

### Generamos las variables para Train, Val y Test

In [ ]:
def preparar_imagenes_para_modelo(data_dict, key_principal='Train', key_imagenes='images'):
    # Extraer imágenes del diccionario
    imagenes = data_dict[key_principal][key_imagenes]

    # Cambiar el orden de las dimensiones a [canales, altura, ancho]
    imagenes = np.transpose(imagenes, (0, 3, 1, 2))

    # Convertir a tensor de PyTorch
    imagenes_tensor = torch.tensor(imagenes, dtype=torch.float32)

    return imagenes_tensor

# Preparar las imágenes para el modelo
Train = preparar_imagenes_para_modelo(data_procesada)
Val = preparar_imagenes_para_modelo(data_procesada, key_principal='Validation')
Test = preparar_imagenes_para_modelo(data_procesada, key_principal='Test')

print('tipo de dato de Train:', Train.type)
print('forma de Train:', Train.shape)
print('forma de Validation:', Val.shape)
print('forma de Test:', Test.shape)

tipo de dato de Train: <built-in method type of Tensor object at 0x7fc2094f3880>
forma de Train: torch.Size([72710, 3, 63, 63])
forma de Validation: torch.Size([500, 3, 63, 63])
forma de Test: torch.Size([500, 3, 63, 63])


### Ahora extraemos las características

In [ ]:
def extraer_etiquetas(data_dict, key_principal='Train', key_etiquetas='labels'):
    # Extraer etiquetas del diccionario
    etiquetas = data_dict[key_principal][key_etiquetas]
    # De etiquetas de diccionario, pasamos a tensor de PyTorch, pindices de clase
    etiquetas_tensor = torch.tensor(etiquetas, dtype=torch.long)
    return etiquetas_tensor

### Extracción de metadata:

In [ ]:
def extraer_metadata(data_dict, key_principal='Train', key_metadata='features'):
    # Extraer características de metadatos del diccionario
    metadata = data_dict[key_principal][key_metadata]
    # Convertir explícitamente a float32
    metadata = metadata.astype(np.float32)
    metadata_tensor = torch.tensor(metadata, dtype=torch.float32)
    return metadata_tensor

# Extraer características de metadatos para los conjuntos de datos
metadata_train = extraer_metadata(data_procesada)
metadata_val = extraer_metadata(data_procesada, key_principal='Validation')
metadata_test = extraer_metadata(data_procesada, key_principal='Test')

print('forma de metadata_train:', metadata_train.shape)
print('forma de metadata_val:', metadata_val.shape)
print('forma de metadata_test:', metadata_test.shape)

metadata_val

forma de metadata_train: torch.Size([72710, 26])
forma de metadata_val: torch.Size([500, 26])
forma de metadata_test: torch.Size([500, 26])


tensor([[ 7.7878e-01, -6.2952e-01,  8.8655e-02,  ..., -1.4237e-01,
         -2.2997e-01, -1.3517e-01],
        [ 7.6781e-01, -6.2212e-01,  1.2532e+00,  ..., -2.5771e-01,
         -3.8337e-01,  9.2198e-02],
        [ 7.5052e-01, -6.1387e-01, -1.0466e+00,  ..., -3.4228e-01,
         -3.9726e-01,  3.2286e-03],
        ...,
        [-1.8353e+00,  2.9948e+00,  8.8655e-02,  ...,  1.6006e-01,
         -2.2666e-01, -5.8001e-01],
        [-1.8213e+00,  1.4213e+00,  8.8655e-02,  ...,  5.7526e-01,
         -3.6618e-01,  6.2541e-02],
        [ 7.7110e-01,  1.4931e+00,  1.2525e+00,  ...,  3.3484e+00,
         -1.3608e-01, -4.7127e-01]])

## Modelo convolucional, incorporando invariancia rotacional. Implementar lo realizado en el paper de referencia

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
import torch

class CNNModel(nn.Module):
    def __init__(self, dropout_p):
        super(CNNModel, self).__init__()
        # Zero padding para evitar perder datos de las imágenes después de las convoluciones
        self.padding = nn.ZeroPad2d(4)  # Ajustado a 5 para compensar el mayor tamaño del kernel
        self.dropout_p = dropout_p

        # Bloques de convolución
        self.conv_blocks = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=0),  # Tamaño del kernel ajustado a 5, padding ajustado
            nn.ReLU(),
            nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=1),  # Tamaño del kernel ajustado a 5, padding ajustado
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),  # Tamaño del kernel ajustado a 5, padding ajustado
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  # Tamaño del kernel ajustado a 5, padding ajustado
            nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1),  # Tamaño del kernel ajustado a 5, padding ajustado
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.BatchNorm2d(64)
        )

        # MLP 1
        self.mlp1 = nn.Sequential(
            nn.Flatten(),
            nn.Linear(16384, 64),
            nn.ReLU(),
        )

        # Normalización de Batch para Metadata
        self.metadata_bn = nn.BatchNorm1d(26)

        self.mlp2 = nn.Sequential(
            nn.Dropout(p=self.dropout_p),
            nn.Linear(90, 64),
            nn.ReLU(),
        )
        self.mlp3 = nn.Sequential(
            nn.Linear(64, 5),
        )

    def _get_conv_output(self, x):
        x = self.padding(x)
        x = self.rotate_input(x)
        x = self.conv_blocks(x)
        return int(np.prod(x.shape[1:]))

    def rotate_input(self, x):
        # Rotaciones para la invariancia
        x_90 = torch.rot90(x, 1, [2, 3])
        x_180 = torch.rot90(x, 2, [2, 3])
        x_270 = torch.rot90(x, 3, [2, 3])
        return torch.cat([x, x_90, x_180, x_270], dim=0)

    def cyclic_pooling(self, x):
        B = x.size(0) // 4
        return (x[:B] + x[B:2*B] + x[2*B:3*B] + x[3*B:]) / 4.0

    def forward(self, x, metadata):
        # Verificación de dimensiones de metadatos
        if metadata.dim() != 2:
            raise ValueError("La metadata debe ser un tensor de 2 dimensiones")
        # Zero padding
        x = self.padding(x)
        # Rotación de la entrada y bloques de convolución
        x_rotated = self.rotate_input(x)
        x_rotated = self.conv_blocks(x_rotated)
        # Primera capa fully connected
        x_rotated_dense = self.mlp1(x_rotated)
        # Pooling Cíclico
        x_pooled = self.cyclic_pooling(x_rotated_dense)
        # Normalización de metadatos
        normalized_metadata = self.metadata_bn(metadata)
        # Concatenación de características de imagen y metadatos
        combined_features = torch.cat([x_pooled, normalized_metadata], dim=1)

        # Pasar por la segunda parte del MLP
        out = self.mlp2(combined_features)
        out = self.mlp3(out)

        return out

    def calculate_linear_input_size(self, input_size):
        """
        Calculates the input size for the first linear layer.

        This function takes a sample input and passes it through the
        convolutional layers to determine the output size, which will
        be the input size for the linear layer.
        """
        x = torch.randn(1, *input_size)  # Create a dummy input
        x = self.padding(x)
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.relu3(x)
        x = self.pool3(x)
        x = x.view(x.size(0), -1)  # Flatten
        self.linear_input_size = x.shape[1]

## Función de entrenamiento y visualización

In [ ]:
def show_curves(curves):
    fig, ax = plt.subplots(1, 2, figsize=(13, 5))
    fig.set_facecolor('white')
    epochs = np.arange(len(curves["val_loss"])) + 1

    ax[0].plot(epochs, curves['val_loss'], label='validation')
    ax[0].plot(epochs, curves['train_loss'], label='training')
    ax[0].set_xlabel('Epoch')
    ax[0].set_ylabel('Loss')
    ax[0].set_title('Loss evolution during training')
    ax[0].legend()

    ax[1].plot(epochs, curves['val_acc'], label='validation')
    ax[1].plot(epochs, curves['train_acc'], label='training')
    ax[1].set_xlabel('Epoch')
    ax[1].set_ylabel('Accuracy')
    ax[1].set_title('Accuracy evolution during training')
    ax[1].legend()

    plt.show()

In [ ]:
def perdida_regularizada_entropia(salidas, objetivos, beta=0.1):
    # Pérdida de entropía cruzada
    perdida_ec = F.cross_entropy(salidas, objetivos, reduction='mean')
    # Calcula la entropía de las probabilidades
    probabilidades = F.softmax(salidas, dim=1)
    entropia = -torch.sum(probabilidades * torch.log(probabilidades + 1e-9), dim=1)
    entropia_reg = torch.mean(entropia)
    # Pérdida combinada
    perdida_total = perdida_ec - beta * entropia_reg

    return perdida_total

In [ ]:
def train_step(x_batch, y_batch, metadata_batch, model, optimizer, criterion, beta):
    # Predicción
    y_predicted = model(x_batch, metadata_batch)
    # Cálculo de la pérdida
    loss = criterion(y_predicted, y_batch, beta=beta)
    # Actualización de parámetros
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    return y_predicted, loss


def evaluate(data_loader, model, criterion, use_gpu):
    cumulative_loss = 0
    cumulative_predictions = 0
    data_count = 0

    for x_val, y_val, metadata_val in data_loader:
        if use_gpu:
            x_val = x_val.cuda()
            y_val = y_val.cuda()
            metadata_val = metadata_val.cuda()
        y_predicted = model(x_val, metadata_val)
        loss = criterion(y_predicted, y_val)
        class_prediction = torch.argmax(y_predicted, axis=1).long()

        cumulative_predictions += (y_val == class_prediction).sum().item()
        cumulative_loss += loss.item()
        data_count += y_val.shape[0]
    val_acc = cumulative_predictions / data_count
    val_loss = cumulative_loss / len(data_loader)

    return val_acc, val_loss

def train_model(
    model,
    train_data_images,
    train_data_labels,
    train_data_metadata,
    val_data_images,
    val_data_labels,
    val_data_metadata,
    epochs,
    criterion,
    batch_size,
    lr,
    use_gpu,
    beta=0.1,
    patience=12,
    val_batch_size=None
):
    if use_gpu:
        model.cuda()

    # Definición de dataloader
    train_dataset = torch.utils.data.TensorDataset(train_data_images, train_data_labels, train_data_metadata)
    val_dataset = torch.utils.data.TensorDataset(val_data_images, val_data_labels, val_data_metadata)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=use_gpu)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=val_batch_size or len(val_data_images), shuffle=False, pin_memory=use_gpu)

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    curves = {
        "train_acc": [],
        "val_acc": [],
        "train_loss": [],
        "val_loss": [],
    }

    t0 = time.perf_counter()
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(epochs):
        print(f"\rEpoch {epoch + 1}/{epochs}")
        cumulative_train_loss = 0
        cumulative_train_corrects = 0
        train_loss_count = 0
        train_acc_count = 0

        # Entrenamiento del modelo
        model.train()

        for i, (x_batch, y_batch, metadata_batch) in enumerate(train_loader):
            if use_gpu:
                x_batch = x_batch.cuda()
                y_batch = y_batch.cuda()
                metadata_batch = metadata_batch.cuda()

            y_predicted, loss = train_step(x_batch, y_batch, metadata_batch, model, optimizer, criterion, beta)

            cumulative_train_loss += loss.item()
            train_loss_count += 1
            train_acc_count += y_batch.shape[0]

            # Calculamos número de aciertos
            class_prediction = torch.argmax(y_predicted, axis=1).long()
            cumulative_train_corrects += (y_batch == class_prediction).sum().item()

            if (i + 1) % 117 == 0:
                print(f"Iteration {i + 1} - Batch {i + 1}/{len(train_loader)} - Train loss: {cumulative_train_loss / train_loss_count}, Train acc: {cumulative_train_corrects / train_acc_count}")

        model.eval()
        with torch.no_grad():
            val_acc, val_loss = evaluate(val_loader, model, criterion, use_gpu)

        print(f"Val loss: {val_loss}, Val acc: {val_acc}")

        train_loss = cumulative_train_loss / train_loss_count
        train_acc = cumulative_train_corrects / train_acc_count
        curves["train_acc"].append(train_acc)
        curves["val_acc"].append(val_acc)
        curves["train_loss"].append(train_loss)
        curves["val_loss"].append(val_loss)

        # Lógica de early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1

        if epochs_without_improvement == patience:
            print(f"Early stopping at epoch {epoch + 1} due to no improvement after {patience} epochs.")
            break

    print(f"Tiempo total de entrenamiento: {time.perf_counter() - t0:.4f} [s]")
    model.cpu()

    return curves

### Buscamos hiperparámetros con funciones de sklearn

In [ ]:
# Dividir las imágenes y metadatos en conjuntos de entrenamiento y validación
Train_images = preparar_imagenes_para_modelo(data_procesada)
Val_images = preparar_imagenes_para_modelo(data_procesada, key_principal='Validation')
Train_labels = extraer_etiquetas(data_procesada)
Val_labels = extraer_etiquetas(data_procesada, key_principal='Validation')
# Extraer características de metadatos para los conjuntos de datos
metadata_train = extraer_metadata(data_procesada)
metadata_val = extraer_metadata(data_procesada, key_principal='Validation')
metadata_test = extraer_metadata(data_procesada, key_principal='Test')

## Aquí empezamos el entrenamiento :) a cambiar hiperparámetros

In [ ]:
!lspci | grep -i nvidia
#Con esta línea, verificamos si hay driver de NVIDIA
#Si existe: use_gpu=True, si no, use_gpu=False

/bin/bash: line 1: lspci: command not found


### Primera prueba de hiperparámetros

In [ ]:
# Instanciación del modelo
lr = 1e-5
dropout_p = 0.5
batch_size = 64
criterion = perdida_regularizada_entropia
epochs = 15
model = CNNModel(dropout_p=dropout_p)

curves = train_model(
    model,
    Train_images,
    Train_labels,
    metadata_train,
    Val_images,
    Val_labels,
    metadata_val,
    epochs,
    criterion,
    batch_size,
    lr,
    use_gpu=False,
    beta=0.4,
    patience=20
)


# Mostrar las curvas de entrenamiento
show_curves(curves)

Epoch 1/15


In [ ]:
def predict(model, data_loader, use_gpu):
    all_preds = []
    all_true = []

    if use_gpu:
        model = model.cuda()  # Transfiere el modelo a la GPU

    model.eval()
    with torch.no_grad():
        for inputs, labels, metadata in data_loader:
            if use_gpu:
                inputs = inputs.cuda()
                metadata = metadata.cuda()

            outputs = model(inputs, metadata)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_true.extend(labels.numpy())

    return all_true, all_preds

# Asegúrate de tener tus datos de test preparados
Test_images = preparar_imagenes_para_modelo(data_procesada, key_principal='Test')
Test_labels = extraer_etiquetas(data_procesada, key_principal='Test')
metadata_test = extraer_metadata(data_procesada, key_principal='Test')

# Definición de dataloader
test_dataset = torch.utils.data.TensorDataset(Test_images, Test_labels, metadata_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

true_labels, predicted_labels = predict(model, test_loader, use_gpu=True)
cm = confusion_matrix(true_labels, predicted_labels)

# Visualizar la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['AGN', 'SN', 'VS', 'asteroid', 'bogus'])
fig, ax = plt.subplots(figsize=(7, 7))  # Ajusta el tamaño del gráfico
disp.plot(cmap=plt.cm.Blues, ax=ax, colorbar=False)  # Elimina la barra de color

# Ajustes adicionales para mejorar la visualización
ax.set_xlabel('Predicted', fontsize=12)
ax.set_ylabel('True', fontsize=12)
ax.grid(False)  # Elimina las líneas de la cuadrícula

plt.xticks(rotation=45)
plt.yticks(rotation=45)

plt.show()

report = classification_report(true_labels, predicted_labels, target_names=['AGN', 'SN', 'VS', 'asteroid', 'bogus'])
print(report)

# Búsqueda de hiperparámetros

In [ ]:
import random

# 1. Define los rangos de los hiperparámetros
param_grid = {
    'dropout_p': [0.2, 0.3, 0.4, 0.5],
    'batch_size': [16, 32, 64],
    'lr': [1e-4, 1e-3, 1e-2, 1e-5],
    'beta': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]
}

# 2. Número de combinaciones a probar
num_combinations = 15

# 3. Realiza la búsqueda aleatoria
best_val_acc = 0
best_params = None
results = []

for i in range(num_combinations):
    print(f"Combinación {i + 1}/{num_combinations}")

    # Genera una combinación aleatoria de hiperparámetros
    params = {k: random.choice(v) for k, v in param_grid.items()}

    # Inicializa el modelo
    model = CNNModel(params['dropout_p'])

    # Entrena el modelo y evalúalo
    curves = train_model(
        model,
        Train_images,
        Train_labels,
        metadata_train,
        Val_images,
        Val_labels,
        metadata_val,
        epochs=20,  # Ajustable
        criterion=perdida_regularizada_entropia,
        batch_size=params['batch_size'],
        lr=params['lr'],
        use_gpu=False,
        beta=params['beta'],
        patience=15
    )


    val_acc = max(curves['val_acc'])

    # Guarda los resultados
    results.append({
        'params': params,
        'val_acc': val_acc,
        'curves': curves,
    })

    # Actualiza los mejores parámetros si es necesario
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_params = params

print("Búsqueda completada")
print(f"Mejor precisión de validación: {best_val_acc}")
print(f"Mejores parámetros: {best_params}")

### Segunda combinación de hiperparámetros

In [ ]:
# Instanciación del modelo
lr = 5e-5
dropout_p = 0.3
batch_size = 64
criterion = perdida_regularizada_entropia
epochs = 15
model = CNNModel(dropout_p=dropout_p)

curves = train_model(
    model,
    Train_images,
    Train_labels,
    metadata_train,
    Val_images,
    Val_labels,
    metadata_val,
    epochs,
    criterion,
    batch_size,
    lr,
    use_gpu=False,
    beta=0.3,
    patience=20
)

# Mostrar las curvas de entrenamiento
show_curves(curves)

In [ ]:
# Asegúrate de tener tus datos de test preparados
Test_images = preparar_imagenes_para_modelo(data_procesada, key_principal='Test')
Test_labels = extraer_etiquetas(data_procesada, key_principal='Test')
metadata_test = extraer_metadata(data_procesada, key_principal='Test')

# Definición de dataloader
test_dataset = torch.utils.data.TensorDataset(Test_images, Test_labels, metadata_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

true_labels, predicted_labels = predict(model, test_loader, use_gpu=True)
cm = confusion_matrix(true_labels, predicted_labels)

# Visualizar la matriz de confusión
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[0, 1, 2, 3, 4])
disp.plot(cmap=plt.cm.Blues)

report = classification_report(true_labels, predicted_labels, target_names=['AGN', 'SN', 'VS', 'asteroid', 'bogus'])

print(report)